In [2]:
!pip install chess

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import chess
import chess.pgn
import chess.svg
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import os
import glob
import time

# Instalar ipywidgets si no está instalado
try:
    import ipywidgets as widgets
except ImportError:
    print("Instalando ipywidgets...")
    import sys
    !{sys.executable} -m pip install ipywidgets
    import ipywidgets as widgets

# Función para mostrar el tablero
def mostrar_tablero(board, titulo=None):
    if titulo:
        print(titulo)
    display(HTML(chess.svg.board(board=board, size=400)))
    print(f"FEN: {board.fen()}")

# Función para cargar una partida desde un archivo PGN
def cargar_partida_desde_archivo(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            juego = chess.pgn.read_game(archivo)
            return juego
    except UnicodeDecodeError:
        # Intentar con otra codificación si UTF-8 falla
        with open(ruta_archivo, 'r', encoding='latin-1') as archivo:
            juego = chess.pgn.read_game(archivo)
            return juego

# Función para reproducir una partida seleccionada
def reproducir_partida(ruta_archivo, output_area, velocidad=1.0):
    # Limpiar el área de salida
    output_area.clear_output()
    
    # Cargar la partida
    with output_area:
        juego = cargar_partida_desde_archivo(ruta_archivo)
        if not juego:
            print(f"No se pudo cargar la partida desde {ruta_archivo}")
            return
        
        # Mostrar información de la partida
        print(f"Evento: {juego.headers.get('Event', 'No especificado')}")
        print(f"Fecha: {juego.headers.get('Date', 'No especificada')}")
        print(f"Blancas: {juego.headers.get('White', 'No especificado')}")
        print(f"Negras: {juego.headers.get('Black', 'No especificado')}")
        print(f"Resultado: {juego.headers.get('Result', 'No especificado')}")
        print("----------------")
        
        # Crear un nuevo tablero
        board = chess.Board()
        
        # Mostrar tablero inicial
        mostrar_tablero(board, "Posición inicial")
        
        # Reproducir los movimientos
        nodo = juego
        while nodo.variations:
            # Obtener el siguiente movimiento
            nodo = nodo.variations[0]
            movimiento = nodo.move
            
            # Aplicar pausa basada en la velocidad
            time.sleep(1.0 / velocidad)
            
            # Obtener la notación SAN antes de realizar el movimiento
            san_notation = board.san(movimiento)
            
            # Realizar el movimiento
            board.push(movimiento)
            
            # Limpiar y mostrar el tablero actualizado
            clear_output(wait=True)
            mostrar_tablero(board, f"Después de {san_notation}")

# Función para buscar archivos PGN en un directorio
def buscar_archivos_pgn(directorio='.'):
    # Buscar todos los archivos PGN (con extensión .pgn o .txt)
    archivos_pgn = glob.glob(os.path.join(directorio, '*.pgn'))
    archivos_pgn.extend(glob.glob(os.path.join(directorio, '*.txt')))
    return archivos_pgn

# Función para extraer información básica de un archivo PGN
def obtener_info_partida(ruta_archivo):
    juego = cargar_partida_desde_archivo(ruta_archivo)
    if juego:
        blancas = juego.headers.get('White', 'No especificado')
        negras = juego.headers.get('Black', 'No especificado')
        evento = juego.headers.get('Event', 'No especificado')
        fecha = juego.headers.get('Date', 'No especificada')
        return f"{blancas} vs {negras} - {evento} ({fecha})"
    return os.path.basename(ruta_archivo)

# Función principal para crear la interfaz de usuario
def crear_selector_partidas(directorio='.'):
    # Obtener archivos PGN
    archivos_pgn = buscar_archivos_pgn(directorio)
    
    if not archivos_pgn:
        print(f"No se encontraron archivos PGN o TXT en {directorio}")
        return
    
    # Crear opciones para el menú desplegable
    opciones = {}
    for archivo in archivos_pgn:
        info = obtener_info_partida(archivo)
        opciones[info] = archivo
    
    # Crear widgets
    dropdown = widgets.Dropdown(
        options=opciones,
        description='Partida:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    slider = widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=5.0,
        step=0.5,
        description='Velocidad:',
        continuous_update=False
    )
    
    boton = widgets.Button(
        description='Reproducir',
        button_style='success',
        tooltip='Haz clic para reproducir la partida'
    )
    
    output_area = widgets.Output()
    
    # Definir el comportamiento del botón
    def on_button_click(b):
        if dropdown.value:
            reproducir_partida(dropdown.value, output_area, slider.value)
    
    boton.on_click(on_button_click)
    
    # Crear la interfaz
    display(widgets.VBox([
        widgets.HBox([dropdown]),
        widgets.HBox([slider, boton]),
        output_area
    ]))

# Para usar el selector, ejecuta:
crear_selector_partidas('/partidas')
# o para usar el directorio actual:
# crear_selector_partidas()